# Projeto 2 - Classificador Automático de Sentimento

Você foi contratado por uma empresa parar analisar como os clientes estão reagindo a um determinado produto no Twitter. A empresa deseja que você crie um programa que irá analisar as mensagens disponíveis e classificará como "relevante" ou "irrelevante". Com isso ela deseja que mensagens negativas, que denigrem o nome do produto, ou que mereçam destaque, disparem um foco de atenção da área de marketing.<br /><br />
Como aluno de Ciência dos Dados, você lembrou do Teorema de Bayes, mais especificamente do Classificador Naive-Bayes, que é largamente utilizado em filtros anti-spam de e-mails. O classificador permite calcular qual a probabilidade de uma mensagem ser relevante dadas as palavras em seu conteúdo.<br /><br />
Para realizar o MVP (*minimum viable product*) do projeto, você precisa implementar uma versão do classificador que "aprende" o que é relevante com uma base de treinamento e compara a performance dos resultados com uma base de testes.<br /><br />
Após validado, o seu protótipo poderá também capturar e classificar automaticamente as mensagens da plataforma.

## Informações do Projeto

Prazo: 13/Set até às 23:59.<br />
Grupo: 1 ou 2 pessoas.<br /><br />
Entregáveis via GitHub: 
* Arquivo notebook com o código do classificador, seguindo as orientações abaixo.
* Arquivo Excel com as bases de treinamento e teste totalmente classificado.

**NÃO disponibilizar o arquivo com os *access keys/tokens* do Twitter.**


### Check 3: 

Até o dia 06 de Setembro às 23:59, o notebook e o xlsx devem estar no Github com as seguintes evidências: 
    * Conta no twitter criada.
    * Produto escolhido.
    * Arquivo Excel contendo a base de treinamento e teste já classificado.
    

Sugestão de leitura:<br />
http://docs.tweepy.org/en/v3.5.0/index.html<br />
https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/

___

## Preparando o ambiente

Instalando a biblioteca *tweepy* para realizar a conexão com o Twitter:

In [28]:
%%capture

#Instalando o tweepy
!pip install tweepy

Importando as Bibliotecas que serão utilizadas. Esteja livre para adicionar outras.

In [29]:
import tweepy
import math
import os.path
import pandas as pd
import json
from random import shuffle

___
## Autenticando no  Twitter

Para realizar a captura dos dados é necessário ter uma conta cadastrada no twitter:

* Conta: ***[Preencha aqui o id da sua conta. Ex: @fulano ]***


1. Caso ainda não tenha uma: https://twitter.com/signup
1. Depois é necessário registrar um app para usar a biblioteca: https://apps.twitter.com/
1. Dentro do registro do App, na aba Keys and Access Tokens, anotar os seguintes campos:
    1. Consumer Key (API Key)
    1. Consumer Secret (API Secret)
1. Mais abaixo, gere um Token e anote também:
    1. Access Token
    1. Access Token Secret
    
1. Preencha os valores no arquivo "auth.pass"

**ATENÇÃO**: Nunca divulgue os dados desse arquivo online (GitHub, etc). Ele contém as chaves necessárias para realizar as operações no twitter de forma automática e portanto é equivalente a ser "hackeado". De posse desses dados, pessoas mal intencionadas podem fazer todas as operações manuais (tweetar, seguir, bloquear/desbloquear, listar os seguidores, etc). Para efeito do projeto, esse arquivo não precisa ser entregue!!!

In [30]:
#Dados de autenticação do twitter:

#Coloque aqui o identificador da conta no twitter: @dudabicalhocd

#leitura do arquivo no formato JSON
with open('auth.pass') as fp:    
    data = json.load(fp)

#Configurando a biblioteca. Não modificar
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])

___
## Coletando Dados

Agora vamos coletar os dados. Tenha em mente que dependendo do produto escolhido, não haverá uma quantidade significativa de mensagens, ou ainda poder haver muitos retweets.<br /><br /> 
Configurando:

In [31]:
#Produto escolhido:
produto = 'Bradesco'

#Quantidade mínima de mensagens capturadas:
n = 500
#Quantidade mínima de mensagens para a base de treinamento:
t = 300

#Filtro de língua, escolha uma na tabela ISO 639-1.
lang = 'pt'

Capturando os dados do twitter:

In [32]:
#Cria um objeto para a captura
api = tweepy.API(auth)

#Inicia a captura, para mais detalhes: ver a documentação do tweepy
i = 1
msgs = []
for msg in tweepy.Cursor(api.search, q=produto, lang=lang).items():    
    msgs.append(msg.text.lower())
    i += 1
    if i > n:
        break

#Embaralhando as mensagens para reduzir um possível viés
shuffle(msgs)

TweepError: Twitter error response: status code = 400

Salvando os dados em uma planilha Excel:

In [33]:
#Verifica se o arquivo não existe para não substituir um conjunto pronto
if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
    #Abre o arquivo para escrita
    writer = pd.ExcelWriter('{0}.xlsx'.format(produto))

    #divide o conjunto de mensagens em duas planilhas
    dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:t])})
    dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

    dfc = pd.DataFrame({'Teste' : pd.Series(msgs[t:])})
    dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

    #fecha o arquivo
    writer.save()

___
## Classificando as Mensagens

Agora você deve abrir o arquivo Excel com as mensagens capturadas e classificar na Coluna B se a mensagem é relevante ou não.<br /> 
Não se esqueça de colocar um nome para a coluna na célula **B1**.<br /><br />
Fazer o mesmo na planilha de Controle.

___
## Montando o Classificador Naive-Bayes

Com a base de treinamento montada, comece a desenvolver o classificador. Escreva o seu código abaixo:

Opcionalmente: 
* Limpar as mensagens removendo os caracteres: enter, :, ", ', (, ), etc. Não remover emojis.<br />
* Corrigir separação de espaços entre palavras e/ou emojis.
* Propor outras limpezas/transformações que não afetem a qualidade da informação.



In [34]:
print(os.getcwd())
dados = pd.read_excel('Bradesco.xlsx')

C:\Users\milen\OneDrive\Documentos\2o SEMESTRE\CDD\Projeto2-CD


In [35]:
#dados

In [36]:
#pegando os dados interessantes para nos
dr= dados.Respostas[dados.Relevancia=='r']
print(len(dr))
probder= 91/300

91


In [37]:
di= dados.Respostas[dados.Relevancia=='i']
print(len(di))
probdei=209/300

209


In [38]:
dicior={}
palavras=[]

# limpando
for x in dr:
    nomes=x.split()
    for y in nomes:
        p2=[]
        for i in y:
            if i!= ":" and i!="'" and i!='"' and i!=' ' and i!= "," and i!= "." and i!='?'and i!='!' and i!='(' and i!=')' :
                p2.append(i)
        a="".join(p2)
        palavras.append(a)
        if y in dicior:
            dicior[y]+=1
        else:
            dicior[y]=1
        
            
#print(dicior)
tr=sum(dicior.values())
print(tr)     
print(palavras)

1475
['já', 'estou', 'cansado', 'de', 'entrar', 'em', 'contato', 'com', 'a', 'ouvidoria', 'do', '@bradesco', 'para', 'solução', 'do', 'meu', 'problema', 'causado', 'pela', 'ineficiência', 'do', 'banco', 'bradesco', 'me', 'ligou', 'oferecendo', 'cartão', 'de', 'crédito', 'e', 'falei', 'que', 'estava', 'desempregado', 'e', 'sem', 'receber', 'renda', 'a', 'atendente', 'desligou', '@bradesco', 'me', 'pedem', 'comprovação', 'alta', 'de', 'renda', 'p/', 'ter', 'o', 'amex', 'tpc', 'p/', 'o', 'cartão', 'negar', 'compras', 'de', 'r$100', 'sugiro', 'por', '1', 'limite', 'visivel', 'pq', 'ta', 'dificil', '@bradesco', 'ja', 'rodei', 'esse', 'app', 'e', 'nao', 'encontro', 'o', 'saldo', 'que', 'investiu', 'se', 'for', 'pra', 'nao', 'ter', 'quero', 'cancelar', 'ne', 'eu', 'dou', 'mas', 'quero', 'receber', 'de', 'volta', 'rs', '@cavacogamer', '@bradesco', 'não', 'demora', 'muito', 'e', 'o', 'bradesco', 'vai', 'falir', 'desse', 'jeito', 'tô', 'tão', 'tisti', 'alguém', 'paga', 'o', 'meu', 'nubank', 'e',

In [39]:
dicioi={}
palavrasi=[]

for x in di:
    nomes=x.split()
    for y in nomes:
        p2=[]
        for i in y:
            if i!= ":" and i!="'" and i!='"' and i!=' ' and i!= "," and i!= "." and i!='?'and i!='!' and i!='(' and i!=')' and i!='…' :
                p2.append(i)
        a="".join(p2)
        palavrasi.append(a)
        if y in dicioi:
            dicioi[y]+=1
        else:
            dicioi[y]=1

#print(dicioi)
ti=sum(dicioi.values())
print(ti)
t=ti+tr  
print(t)
print(palavrasi)

2966
4441
['bandidos', 'explodem', 'caixas', 'do', 'bradesco', 'em', 'prédio', 'da', 'prefeitura', 'de', 'campina', 'grande', 'no', 'centro', 'da', 'cidade', 'https//tco/pwv2snxcct', 'rt', '@magnopb', 'bandidos', 'explodem', 'caixas', 'do', 'bradesco', 'em', 'prédio', 'da', 'prefeitura', 'de', 'campina', 'grande', 'no', 'centro', 'da', 'cidade', 'https//tco/b4xlbmp4w0', 'h', 'im', 'at', 'bradesco', 'in', 'são', 'paulo', 'sp', 'https//tco/kesmfnei9k', '@undeadstef', 'noooosssa', 'pode', 'colocar', 'bradesco', 'nessa', 'lista', 'também', 'acho', 'que', 'nenhum', 'presta', 'mesmo', '', '@bradesco', 'não', 'caso', 'o', 'banco', 'tenha', 'enviado', 'foi', 'para', 'o', 'meu', 'antigo', 'endereço', 'rt', '@umvesgo', 'e', 'até', 'quem', 'me', 'vê', 'na', 'fila', 'do', 'bradesco', 'sabe', 'que', 'eu', 'te', 'encontrei', '@victorissso', 'infelizmente', 'sim', 'recebi', 'um', 'email', 'hj', 'acho', 'q', 'só', 'bradesco', 'bandeira', 'visa', 'ou', 'seja', 'meu', 'cartão', '@fabioagitadas', 'no', '

In [40]:
#probabilidade da frase ser relavante e relevante=
#(vezes q a palavra aparece+1)/(palavras relevantes+total de palavras) x a probab da outra palavra em relevante x ....
#qual eh maior
pfr=1
frase="bradesco me cansa"
nomes=frase.split()
for i in nomes:
    if i in dicior:
        pa=(dicior[i]+1)/(tr+t)
        pfr*=pa
    else:
        pa=1/(tr+t)
        pfr*=pa

print(pfr)

1.7386733795816556e-09


In [41]:
pfi=1
frase="bradesco me cansa"
for i in frase:
    if i in dicioi:
        pa= (dicioi[i]+1)/(tr+t)
        pfr*=pa
    else:
        pa=1/(tr+t)
        pfi*=pa

print(pfi)

3.218771752973153e-42


In [42]:
if pfi<pfr:
    print('É relevante')
else:
    print('É irrelevante')

É relevante


___
## Verificando a performance

Agora você deve testar o seu Classificador com a base de Testes.<br /><br /> 

Você deve extrair as seguintes medidas:
* Porcentagem de positivos falsos (marcados como relevante mas não são relevantes)
* Porcentagem de positivos verdadeiros (marcado como relevante e são relevantes)
* Porcentagem de negativos verdadeiros (marcado como não relevante e não são relevantes)
* Porcentagem de negativos falsos (marcado como não relevante e são relevantes)

Opcionalmente:
* Criar categorias intermediárias de relevância baseado na diferença de probabilidades. Exemplo: muito relevante, relevante, neutro, irrelevante e muito irrelevante.

In [48]:
#fazer uma nov coluna e comparar as colunas
print(os.getcwd())
dados2 = pd.read_excel('Bradesco.xlsx', 1)

C:\Users\milen\OneDrive\Documentos\2o SEMESTRE\CDD\Projeto2-CD


In [49]:
dados2.head()

,Teste,Tweets,Relevancia
0,1,"só o bradesco que n vai ter feriadão, q bosta",i
1,2,novidades bradesco saúde. https://t.co/ndk1na7vag,i
2,3,são 84 cursos grátis e com certificado da fund...,i
3,4,@bradesco cheguei 12:05h agência cachambi em f...,i
4,5,rt @bradesco: a história de orgulho dos gaúcho...,i


___
## Concluindo

Escreva aqui a sua conclusão.<br /> 
Faça um comparativo qualitativo sobre as medidas obtidas.<br />
Explique como são tratadas as mensagens com dupla negação e sarcasmo.<br />
Proponha um plano de expansão. Por que eles devem continuar financiando o seu projeto?<br />

Opcionalmente: 
* Discorrer por que não posso alimentar minha base de Treinamento automaticamente usando o próprio classificador, aplicado a novos tweets.
* Propor diferentes cenários de uso para o classificador Naive-Bayes. Cenários sem intersecção com este projeto.
* Sugerir e explicar melhorias reais no classificador com indicações concretas de como implementar (não é preciso codificar, mas indicar como fazer e material de pesquisa sobre o assunto).
